![Natural Earth logo](https://www.naturalearthdata.com/wp-content/themes/NEV/images/nev_logo.png "Natural Earth logo")
<div align="center">

## Scraping bodies of water data
</div>

Document explaining how bodies of water data is processed and saved in PostgreSQL database.

Link to data from Natural Earth: [bodies of water data](https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_geography_marine_polys.zip)

<hr>

# 1. Download and load data

## Data is in shapefile format compressed into zip file. There is need to import geopandas to read spatial data and requests to get zip file from web.

In [ ]:
import geopandas as gpd
import requests
import os

In [ ]:
waters_link = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_geography_marine_polys.zip"

## Get zip file and check status code. 200 is OK.

In [ ]:
r = requests.get(waters_link ,stream=True, headers={"User-Agent": "XY"})
r.status_code

## Save zip file inside temp folder.

In [ ]:
if not os.path.exists("../temp/waters/"):
    os.makedirs("../temp/waters/")

In [ ]:
with open('../temp/waters/ne_10m_geography_marine_polys.zip', 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
zip_file = "zip://../temp/waters/ne_10m_geography_marine_polys.zip!ne_10m_geography_marine_polys.shp"

## Load shapefile into geopandas dataframe.

In [ ]:
waters_shp = gpd.read_file(
    zip_file, layer='ne_10m_geography_marine_polys'
)

## Check number of row, columns and crs shapefile data.

In [ ]:
waters_shp.shape

In [ ]:
waters_shp.crs

## Modify dataframe to display only columns that are valuable for our dataset.

In [ ]:
waters_shp.columns.values.tolist()

In [ ]:
filter_waters = waters_shp[
    [
        "featurecla",
        "name",
        "note",
        "wikidataid",
        "ne_id",
        "geometry",
    ]
]

In [ ]:
filter_waters

# 2. Split and view avaliable data

## Check body of water types

In [ ]:
water_types = set(filter_waters["featurecla"])

In [ ]:
water_types

## Filter and view all bays

In [ ]:
filter_bay = filter_waters["featurecla"].eq("bay")

In [ ]:
filter_waters[filter_bay].explore(popup=True)

## Filter and view all channels

In [ ]:
filter_channel = filter_waters["featurecla"].eq("channel")

In [ ]:
filter_waters[filter_channel].explore(popup=True)

## Filter and view all fjords

In [ ]:
filter_fjord = filter_waters["featurecla"].eq("fjord")

In [ ]:
filter_waters[filter_fjord].explore(popup=True)

## Filter and view all generic

In [ ]:
filter_generic = filter_waters["featurecla"].eq("generic")

In [ ]:
filter_waters[filter_generic].explore(popup=True)

## Filter and view all gulfs

In [ ]:
filter_gulf = filter_waters["featurecla"].eq("gulf")

In [ ]:
filter_waters[filter_gulf].explore(popup=True)

## Filter and view all inlets

In [ ]:
filter_inlet = filter_waters["featurecla"].eq("inlet")

In [ ]:
filter_waters[filter_inlet].explore(popup=True)

## Filter and view all lagoons

In [ ]:
filter_lagoon = filter_waters["featurecla"].eq("lagoon")

In [ ]:
filter_waters[filter_lagoon].explore(popup=True)

## Filter and view all oceans

In [ ]:
filter_ocean = filter_waters["featurecla"].eq("ocean")

In [ ]:
filter_waters[filter_ocean].explore(popup=True)

## Filter and view all reefs

In [ ]:
filter_reef = filter_waters["featurecla"].eq("reef")

In [ ]:
filter_waters[filter_reef].explore(popup=True)

## Filter and view all rivers

In [ ]:
filter_river = filter_waters["featurecla"].eq("river")

In [ ]:
filter_waters[filter_river].explore(popup=True)

## Filter and view all seas

In [ ]:
filter_sea = filter_waters["featurecla"].eq("sea")

In [ ]:
filter_waters[filter_sea].explore(popup=True)

## Filter and view all sounds

In [ ]:
filter_sound = filter_waters["featurecla"].eq("sound")

In [ ]:
filter_waters[filter_sound].explore(popup=True)

## Filter and view all straits

In [ ]:
filter_strait = filter_waters["featurecla"].eq("strait")

In [ ]:
filter_waters[filter_strait].explore(popup=True)

# 3. Save data in database

In [ ]:
from apps.utils.converters import pandas_to_gis_multipolygon
from apps.landform.models import BodyOfWater, BodyOfWaterType

## Create body of water FK

In [ ]:
[BodyOfWaterType.objects.get_or_create(name=item) for item in water_types if item]

## Iterate through body of water dataframe, convert data, get FK , update or create new entry in database.

In [ ]:
for row in filter_waters.itertuples(index=False, name="Pandas"):

    geometry = pandas_to_gis_multipolygon(row.geometry)
    body_of_water_type = BodyOfWaterType.objects.filter(name=row.featurecla)
    body_of_water_type = body_of_water_type[0] if body_of_water_type else None

    updated_values = {
        "geometry": geometry,
        "wikidata_id": row.wikidataid,
        "body_of_water_type": body_of_water_type,
        "name": row.name,
    }

    BodyOfWater.objects.update_or_create(
        natural_earth_id=row.ne_id,
        defaults=updated_values,
    )